In [16]:
#sys.path.append("/root/anaconda3/Lib/site-packages/MLC")
import MLC.MLC as gp
import gymnasium as gym
import random
import numpy as np
import pde
import copy
import math

In [18]:
#to make gymanasium work with py-pde, need to pass in action step:
#1.  control function to calculate control with all values constant except x,y
#2.  grid
#controller servers to pass these values to gymnasium.
#control will be nxm vector, where n is num_controls and m is number of cells per boundary
#if non-square boundary, m will be larger side and in gymnasium will need to hard code truncation of extra cells

class controller:
    def __init__(self, grid, state, num_sens):
        self.grid=grid
        self.num_sens=num_sens
        self.state=state
        self.control=[]

#need function to reduce full space to just sensors 
def sensor_meas(sensors,domain):
    #assume senors form a square grid, so take square root of number of sensors
    #assume sensors read from middle of domain
    n_sense=int(np.sqrt(sensors))
    meas=np.empty((n_sense,n_sense))
    dimx=domain.shape[0]
    dimy=domain.shape[1]
    startx=round((dimx-n_sense)/2)
    starty=round((dimy-n_sense)/2)
    for i in range(n_sense):
        for j in range(n_sense):
            meas[i,j]=float(domain[i+startx,j+starty])
    return meas.flatten()

In [ ]:
env = gym.make("Diffusion-v0")
observation, grid, state = env.reset(seed=412)

#####notes on using gymnasium with py-pde######
#observation needs to be full grid area, but number of sensors will probably be less

#array with target values for sensors

Ni=25 #number of individuals
Np=25 #idnvidiuals selected for advancement tournament each round
Ne=5  #best Ne of each forest advanced unchanged
Nn=0.5 #for reseed, precent reseeded each round

Num_sens=4*4 #size of observation/sensor space in number of grid points
grid_dim=2 #x,y for including cotnrol location in alogrithim
Ns=grid_dim+Num_sens #sensors are first in s array, then 

Pr=0.1 #probably of replication
Pm=0.3 #probably of mutation
Pc=0.6 #probably of crossover

maxi=15 #set max number of iterations

#define controls: number of controls, control locations.  Assume control is on boundary
#here, with two control locations and 4 controls, that means that two boundaries have controls.
#In gymanisum this will be hard coded as bottom and right side, or (x,ymin) and (xmax, y)
num_control = 4
control_size = 3 #number of grid points comprising controller
control_points = np.array([8,20]) #grid x/y locations of controllers
dx=1.
dy=1.
xmax=grid.shape[0]*dx
ymax=grid.shape[1]*dy 

implement=controller(grid, state, Num_sens)

#hard code control locations
control_loc=np.array([xmax, dy*control_points[0]]) #right side, 1st control point
control_loc=np.vstack([control_loc,[xmax, dy*control_points[1]]]) #right side, 2nd control point
control_loc=np.vstack([control_loc,[ymax, dx*control_points[0]]]) #top, 1st control point
control_loc=np.vstack([control_loc,[ymax, dx*control_points[1]]]) #top 2nd control point
#controller will be defined by the control vectors passed to py-pde via gymnasium.  Wherever controllers are set (probably here)
#controller location variable is only half of controller location.  the other is which boundaries.  
#can do different location along each boundary or same.  Here, for simplicity, do same along each boundary that has a control.  
#Controls are just boundary conditions on the boundary controls
#however, gymnaisum will have hard-coded which boundaries have controls--this will get "value" as an array

#Generally, implementing control requires planning and some hard coding.  User must know before coding controller how many gridpoints
#are on each boundary with a controller, decide what boundary(ies) to put controller on
#how many control points on the boundary(ies), how many girdpoints each control point uses, if control points are at same location
#on each boundary, and, below, the x/y coordinates for each control point to calculate the control input arries for each boundary.
#the control is implemented by an array with number of values equal to the grid points on the associated boundary.  
#Recommended for easier implementation to make control type same type as boundary layer (i.e. dirchliet vs neumann)
#array will have boundary condition (say, 0 dirchlet) everywhere except the control grid points, which will have value
#determined by algorithm at control point and constant over grid points matching control size.  
#also need to know real extents of boundary--i.e. is each cell 1 unit or 1/32 unit, etc. via dx, dy which is hard coded based on grid design
#i.e. if a right boundary is 32 grid points tall and control location is halfway up the boundary (location 16) with size of 3
#then control will be calculated based on xmax, ymax/2, sensor readings and then applied at cells 15, 16, 17

#need to get initial measurements from observation, fortunately already is a numpy array (!)
#now this is less than Ns (and so then number of variables), remaineders are x,y location for all controllers (num_control)
s=sensor_meas(Num_sens,state.data)
#
K=np.empty(num_control) #dummy initialization of K

forest=[]
#generate forest of Ni trees
for i in range(0,Ni):
    seed=gp.sapling(3,8,Ns)
    trial=gp.growTrig(0,2,seed)
    forest.append(seed)

minJ=50000 #set initial "best" cost
k=0 #iteration counter

while (minJ>400 and k<maxi):
    #make random starting point each forest generation, but same for each tree per generation
    n=random.randint(0,10000)
    subtotal=0
    for t in forest:
        #reset the environment for each tree using same enivornment for each tree with each time step
        observation, grid, state = env.reset(seed=n)
        #iterate through a list of trees and store best
        #to determine cost, need K and reward for every timestep, env.step provides
        #make sure reward is reset in case tree is reused
        t.total=0
        s=sensor_meas(Num_sens,state.data)
        #need to take a pass of '500' time steps through environment each tree, each iteration
        #for inverted pendulum need to try on several different seeds becuase getting lucky shots that work for one starting point
        for i in range(50):
            control_name=t.name
            #need to replace each "sensor" value with updated observation
            #will need to also replace x and y values for controller locations, which result in a vector of results
            #one element per controller
            #probably have each controller operate across several cells
            #best to just make an array of control inputs here or in gym and then push to py-pde boundary as value

            #to make control/boundary array, first update algorthim variables with sensor values (1:n-2 variables)
            for j in range(Num_sens):
                obs_name='s'+str(j)
                control_name=[s[j].item() if c==obs_name else c for c in control_name]
                
            #then make num_control different vectors, each with different x value
            for j in range(control_loc.shape[0]):
                holder=copy.copy(control_name)
                for k in range(Num_sens, Ns):
                    obs_name='s'+str(k)
                    control_name=[control_loc[j,k-Num_sens].item() if c==obs_name else c for c in control_name]
                k,dummy=gp.evaluate(control_name)
                if k>10:  k=10.0
                elif k<-10:  k=-10.0
                K[j]=k

            #now set up entire array to pass for the boundary
            #hard coded that non-control points will be 0, control points will be as appropriate for that control value
            #need to keep order of boundaries straight for K, for passing to gymnasium, and in gymnasium
            left_bound=np.zeros(grid.shape[1])
            right_bound=np.zeros(grid.shape[1])
            bottom_bound=np.zeros(grid.shape[0])
            top_bound=np.zeros(grid.shape[0])
            for i in range(len(control_points)):
                start=math.floor(control_size/2)
                right_bound[control_points[i]-start:control_points[i]-start+control_size]=K[i]
                top_bound[control_points[i]-start:control_points[i]-start+control_size]=K[i+2]
            all_bound=np.vstack([left_bound, right_bound])
            all_bound=np.vstack([all_bound, bottom_bound])
            all_bound=np.vstack([all_bound, top_bound])

            implement.control=all_bound
            
            obs, reward, terminated, truncated, info = env.step(implement)
            implement.state.data=obs
            subtotal+=reward
        t.total=subtotal
        if t.total<minJ: 
            minJ=t.total
            bestTree=t.name
            print(i, minJ, bestTree)
#either "tournament" or reseed random half of the forest
    #uncomment below for reseed random half of forest
        env.close()
    forest=gp.evolution_reseed(Nn, Ni, Ne,Pc, Pm, forest)

    #uncomment below for tournament approach
    #need to choose Np individuals for each selection round. 
    # next_forest=selection(Ni, Np, forest)
    # for t in next_forest:
    #     if len(t.name)==0: print("selection empty")
    #     if type(t.name)==bool:  print("selection bool")
    #Np individuals will be ranked and the best will advance to replication, mutation, crossover
    # forest=evolution(Ne, Pc, Pm, next_forest)
    # for i in range(0, Ni):
    #     print(forest[i].name)
    k+=1
    print(k, minJ, bestTree)

print("best:  ",minJ, bestTree, k)


C:\Users\Robert\anaconda3\Lib\site-packages\gymnasium\utils\passive_env_checker.py:193: UserWarning: WARN: The result returned by `env.reset()` should be `(obs, info)` by default, , where `obs` is a observation and `info` is a dictionary containing additional information.
  logger.warn(
Exception ignored in: <function ObjectRef.__del__ at 0x00000253EF4C8FE0>
Traceback (most recent call last):
  File "C:\Users\Robert\anaconda3\Lib\site-packages\llvmlite\binding\ffi.py", line 375, in __del__
    self.close()
  File "C:\Users\Robert\anaconda3\Lib\site-packages\llvmlite\binding\ffi.py", line 335, in close
    self._dispose()
  File "C:\Users\Robert\anaconda3\Lib\site-packages\llvmlite\binding\module.py", line 77, in _dispose
    self._capi.LLVMPY_DisposeModule(self)
  File "C:\Users\Robert\anaconda3\Lib\site-packages\llvmlite\binding\ffi.py", line 192, in __call__
    return self._cfn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
OSError: exception: access violation reading 0x0000

In [ ]:
#try without gymnasium, just going through whole PDE with each equation
#need to modify to make multiple outputs or multiple trees...

Ni=500 #number of individuals
Np=25 #idnvidiuals selected for advancement tournament each round
Ne=5  #best Ne of each forest advanced unchanged
Nn=0.5 #for reseed, precent reseeded each round
Ns=16+2    #list(env.observation_space.shape)[0] #number of sensors

Pr=0.1 #probably of replication
Pm=0.3 #probably of mutation
Pc=0.6 #probably of crossover

maxi=505 #set max number of iterations

#need to get initial measurements from observation, fortunately already is a numpy array (!)
s=observation
b=0 #dummy initialization of b

#will need to refactor for "online" programming, rather than having all measurments avilable at once

forest=[]
#generate forest of Ni trees
for i in range(0,Ni):
    seed=gp.sapling(2,7,Ns)
    trial=gp.growTrig(0,2,seed)
    forest.append(seed)

minJ=-50000 #set initial "best" cost
k=0 #iteration counter

#initialize grid for PDE solver
grid = pde.UnitGrid([64, 64], periodic=[False, False])    

while (minJ<-4900 and k<maxi):
    #make random starting point each forest generation, but same for each tree per generation
    n=random.randint(0,10000)
    for t in forest:
        #reset the environment for each tree using same enivornment for each tree with each time step
        state = pde.ScalarField.random_uniform(grid)
        #make sure reward is reset in case tree is reused
        t.total=0
        s=observation

        for i in range(500):
            control_name=t.name
            #need to replace each "sensor" value with updated observation
            for j in range(len(s)):
                obs_name='s'+str(j)
                control_name=[s[j].item() if c==obs_name else c for c in control_name]
            K,dummy=gp.evaluate(control_name)
            if K>1:  K=1.0
            elif K<-1:  K=-1.0
            K=np.asarray([K])
            obs, reward, terminated, truncated, info = env.step(K)
            if reward>10:  print(reward, i, n, obs)
            t.total+=-1.*reward#+len(control_name)/4.
            if terminated:  t.total+=10000 #include for inverted pendulum
            s=obs
            if (terminated or truncated):  break
        t.total=t.total#/(i+1)
        if t.total<minJ: 
            minJ=t.total
            bestTree=t.name
            print(i, minJ, bestTree)
#either "tournament" or reseed random half of the forest
    #uncomment below for reseed random half of forest
        env.close()
    forest=gp.evolution_reseed(Nn, Ni, Ne,Pc, Pm, forest)

    #uncomment below for tournament approach
    #need to choose Np individuals for each selection round. 
    # next_forest=selection(Ni, Np, forest)
    # for t in next_forest:
    #     if len(t.name)==0: print("selection empty")
    #     if type(t.name)==bool:  print("selection bool")
    #Np individuals will be ranked and the best will advance to replication, mutation, crossover
    # forest=evolution(Ne, Pc, Pm, next_forest)
    # for i in range(0, Ni):
    #     print(forest[i].name)
    k+=1
    print(k, minJ, bestTree)

print("best:  ",minJ, bestTree, k)